In [2]:
import xml.etree.ElementTree as ET

In [65]:
qa = ET.parse('../data/CRF_training/qa_test.xml')
root = qa.getroot()

In [15]:
pair = []
B, I, O = 'B-ANS', 'I-ANS', 'O-ANS'

count_qa = 1
count_cand = 1
for qa_pair in root:
    question = qa_pair.find('question')
    question_text = question.text.split('\n')[1].strip().split()
    question_pos = question.text.split('\n')[2].strip().split()
    question_ner = question.text.split('\n')[3].strip().split()
    question_qword = question.text.split('\n')[4].strip()
    question_qtype = question.text.split('\n')[5].strip()
    question_comparisons = question.text.split('\n')[6].strip().split(',')
    question_comparisons = list(map(lambda item: item.strip(), question_comparisons))
    question_delimiters = question.text.split('\n')[7].strip().split(',')
    question_delimiters = list(map(lambda item: item.strip(), question_delimiters))
    question_headwords = question.text.split('\n')[8].strip().split(',')
    question_headwords = list(map(lambda item: item.strip(), question_headwords))
    
    print(question.text.split('\n')[1].strip())
    
    crf_line = []
    # For candidate
    # pos[-2], pos[-2]|pos[-1], pos[-2]|pos[-1]|pos[0], pos[0], pos[0]|pos[1], pos[0]|pos[1]|pos[2]
    # ner[-2], ner[-2]|ner[-1], ner[-2]|ner[-1]|ner[0], ner[0], ner[0]|ner[1], ner[0]|ner[1]|ner[2]
    # qword
    # atype
    # qcomp[1], qcomp[2]
    # qdel[1], qdel[0]
    # WMD
    # qheadword
    # B-ANS, I-ANS, O-ANS
    
    for candidate in qa_pair.findall('candidate'):
        text = candidate.text.split('\n')[1].strip().split()
        pos = candidate.text.split('\n')[2].strip().split()
        ner = candidate.text.split('\n')[3].strip().split()
        distance = candidate.text.split('\n')[4].strip()
        answers = candidate.text.split('\n')[5].strip().split(',')
        answers = list(map(lambda item: item.strip(), answers))
        
        # Extract token for answer
        answer_tagged = []
        for answer in answers:
            if answer != '-':
                answer_tokens = answer.split()
                answer_tagged = [(answer_tokens[0], B)]
                answer_tagged.extend([(token, I) for token in answer_tokens[1:]])
        
        print(f'QAPair {count_qa}')
        print(f'\tCandidate {count_cand}: {answer_tagged}')
        
        dict_tagged = dict(answer_tagged)
        for index, text_token in enumerate(text):
            if answer_tagged:
                if text_token in dict_tagged.keys():
                    crf_line.append([dict_tagged[text_token], extract_pos_from_candidate(text, pos, (index, text_token)), 
                                     extract_ner_from_candidate(text, ner, (index, text_token)), f'wdm={distance}', extract_question_features(question_qword, question_text, question_qtype, question_pos, question_ner, question_headwords, question_comparisons, question_delimiters)])
                else:
                    crf_line.append([O, extract_pos_from_candidate(text, pos, (index, text_token)),
                                     extract_ner_from_candidate(text, ner, (index, text_token)), f'wdm={distance}', extract_question_features(question_qword, question_text, question_qtype, question_pos, question_ner, question_headwords, question_comparisons, question_delimiters)])
                    
            else:
                crf_line.append([O, extract_pos_from_candidate(text, pos, (index, text_token)),
                                 extract_ner_from_candidate(text, ner, (index, text_token)), f'wdm={distance}', extract_question_features(question_qword, question_text, question_qtype, question_pos, question_ner, question_headwords, question_comparisons, question_delimiters)])
        
        print(f'\tCandidate {count_cand}: {crf_line}')
        count_cand += 1
    count_qa += 1
        

which course has been viewed by MMF ANB MU SA KK ?
QAPair 1
	Candidate 1: [('CS260', 'B-ANS'), ('Human', 'I-ANS'), ('Computer', 'I-ANS'), ('Interaction', 'I-ANS')]
	Candidate 1: [['O-ANS', 'pos[0]=NNP\tpos[0]|pos[1]=NNP|VBD\tpos[0]|pos[1]|pos[2]=NNP|VBD|TO', 'ner[0]=LogUser\tner[0]|ner[1]=LogUser|LogAction\tner[0]|ner[1]|ner[2]=LogUser|LogAction|Other', 'wdm=0.8309', 'qword|qword_pos|qword_ner=which|WDT|Other\tqtype=LogResource\tqheadword[0]=course\tqheadword_pos[0]=NN\tqheadword_ner[0]=LogInformation\tqdelimiter[0]=by\tqdelimiter_pos[0]=IN\tqdelimiter_ner[0]=Other'], ['O-ANS', 'pos[-1]=NNP\tpos[-1]|pos[0]=NNP|VBD\tpos[-1]|pos[0]|pos[1]=NNP|VBD|TO', 'ner[-1]=LogUser\tner[-1]|ner[0]=LogUser|LogAction\tner[-1]|ner[0]|ner[1]=LogUser|LogAction|Other', 'wdm=0.8309', 'qword|qword_pos|qword_ner=which|WDT|Other\tqtype=LogResource\tqheadword[0]=course\tqheadword_pos[0]=NN\tqheadword_ner[0]=LogInformation\tqdelimiter[0]=by\tqdelimiter_pos[0]=IN\tqdelimiter_ner[0]=Other'], ['O-ANS', 'pos[-2]=NNP\

ValueError: ' least' is not in list

In [11]:
def extract_pos_from_candidate(text, pos, word_index):
    """
        Utility function used to extract chunking pos for a word
    """
    
    if len(text) != len(pos):
        print(text)
        print(pos)
        print(len(text), len(pos))
        sys.exit(1)
    
    index, word = word_index[0], word_index[1]
    
    if index == 0:
        return f'pos[0]={pos[0]}\tpos[0]|pos[1]={pos[0]}|{pos[1]}\tpos[0]|pos[1]|pos[2]={pos[0]}|{pos[1]}|{pos[2]}'
    elif  index == 1:
        return f'pos[-1]={pos[index-1]}\tpos[-1]|pos[0]={pos[index-1]}|{pos[index]}\tpos[-1]|pos[0]|pos[1]={pos[index-1]}|{pos[index]}|{pos[index+1]}'
    elif index == len(text) - 1:
        return f'pos[-2]={pos[index-2]}\tpos[-2]|pos[-1]={pos[index-2]}|{pos[index-1]}\tpos[-2]|pos[-1]|pos[0]={pos[index-2]}|{pos[index-1]}|{pos[index]}'
    elif index == len(text) - 2:
        return f'pos[-2]={pos[index-2]}\tpos[-2]|pos[-1]={pos[index-2]}|{pos[index-1]}\tpos[-2]|pos[-1]|pos[0]={pos[index-2]}|{pos[index-1]}|{pos[index]}\tpos[0]={pos[index]}\tpos[0]|pos[1]={pos[index]}|{pos[index+1]}'
    else:
        return f'pos[-2]={pos[index-2]}\tpos[-2]|pos[-1]={pos[index-2]}|{pos[index-1]}\tpos[-2]|pos[-1]|pos[0]={pos[index-2]}|{pos[index-1]}|{pos[index]}\tpos[0]={pos[index]}\tpos[0]|pos[1]={pos[index]}|{pos[index+1]}\tpos[0]|pos[1]|pos[2]={pos[index]}|{pos[index+1]}|{pos[index+2]}'


def extract_ner_from_candidate(text, ner, word_index):
    """
        Utility function used to extract chunking ner for a word
    """
    
    if len(text) != len(ner):
        print(text)
        print(ner)
        print(len(text), len(ner))
        sys.exit(1)
    
    index, word = word_index[0], word_index[1]
    
    if index == 0:
        return f'ner[0]={ner[0]}\tner[0]|ner[1]={ner[0]}|{ner[1]}\tner[0]|ner[1]|ner[2]={ner[0]}|{ner[1]}|{ner[2]}'
    elif  index == 1:
        return f'ner[-1]={ner[index-1]}\tner[-1]|ner[0]={ner[index-1]}|{ner[index]}\tner[-1]|ner[0]|ner[1]={ner[index-1]}|{ner[index]}|{ner[index+1]}'
    elif index == len(text) - 1:
        return f'ner[-2]={ner[index-2]}\tner[-2]|ner[-1]={ner[index-2]}|{ner[index-1]}\tner[-2]|ner[-1]|ner[0]={ner[index-2]}|{ner[index-1]}|{ner[index]}'
    elif index == len(text) - 2:
        return f'ner[-2]={ner[index-2]}\tner[-2]|ner[-1]={ner[index-2]}|{ner[index-1]}\tner[-2]|ner[-1]|ner[0]={ner[index-2]}|{ner[index-1]}|{ner[index]}\tner[0]={ner[index]}\tner[0]|ner[1]={ner[index]}|{ner[index+1]}'
    else:
        return f'ner[-2]={ner[index-2]}\tner[-2]|ner[-1]={ner[index-2]}|{ner[index-1]}\tner[-2]|ner[-1]|ner[0]={ner[index-2]}|{ner[index-1]}|{ner[index]}\tpos[0]={ner[index]}\tner[0]|ner[1]={ner[index]}|{ner[index+1]}\tner[0]|ner[1]|ner[2]={ner[index]}|{ner[index+1]}|{ner[index+2]}'
    
    
def extract_question_features(qword, qtext, qtype, qpos, qner, qheadwords, qcomps, qdels):
    """
        Utility function used to extract question related feature
    """
    
    features = []
    headwords = ''
    headwords_pos = ''
    headwords_ner = ''
    
    comparisons = ''
    comparisons_pos = ''
    comparisons_ner = ''
    
    delimiters = ''
    delimiters_pos = ''
    delimiters_ner = ''
    
    qword_pos = qpos[qtext.index(qword)]
    qword_ner = qner[qtext.index(qword)]
    qwords = f'qword|qword_pos|qword_ner={qword}|{qword_pos}|{qword_ner}'
    
    names = []
    values = []
    names_pos = []
    values_pos = []
    names_ner = []
    values_ner =[]
    
    features.append(qwords)
    features.append(f'qtype={qtype}')
    
    h_pos = ''
    h_ner = ''
    
    for index, headword in enumerate(qheadwords):
        names.append(f'qheadword[{index}]')
        values.append(f'{headword}')
        headwords = f'{"|".join(names)}={"|".join(values)}'
        
        names_pos.append(f'qheadword_pos[{index}]')
        if len(headword.split()) > 1:
            h_pos = ' '.join([qpos[qtext.index(h)] for h in headword.split()])
        values_pos.append(f'{h_pos}')
        headwords_pos = f'{"|".join(names_pos)}={"|".join(values_pos)}'
        
        names_ner.append(f'qheadword_ner[{index}]')
        if len(headword.split()) > 1:
            h_ner = ' '.join([qner[qtext.index(h)] for h in headword.split()])
        values_ner.append(f'{h_ner}')
        headwords_ner = f'{"|".join(names_ner)}={"|".join(values_ner)}'
    
    features.append(headwords)
    features.append(headwords_pos)
    features.append(headwords_ner)
    
    names = []
    values = []
    names_pos = []
    values_pos = []
    names_ner = []
    values_ner = []
    
    if '-' not in qcomps:
        for index, comp in enumerate(qcomps):
            names.append(f'qcomp[{index}]')
            values.append(f'{comp}')
            comparisons = f'{"|".join(names)}={"|".join(values)}'

            names_pos.append(f'qcomp_pos[{index}]')
            values_pos.append(f'{qpos[qtext.index(comp)]}')
            comparisons_pos = f'{"|".join(names_pos)}={"|".join(values_pos)}'

            names_ner.append(f'qcomp_ner[{index}]')
            values_ner.append(f'{qner[qtext.index(comp)]}')
            comparisons_ner = f'{"|".join(names_ner)}={"|".join(values_ner)}'

        features.append(comparisons)
        features.append(comparisons_pos)
        features.append(comparisons_ner)

        names = []
        values = []
        names_pos = []
        values_pos = []
        names_ner = []
        values_ner = []
    
    if '-' not in qdels:
        for index, delimiter in enumerate(qdels):
            names.append(f'qdelimiter[{index}]')
            values.append(f'{delimiter}')
            delimiters = f'{"|".join(names)}={"|".join(values)}'

            names_pos.append(f'qdelimiter_pos[{index}]')
            values_pos.append(f'{qpos[qtext.index(delimiter)]}')
            delimiters_pos = f'{"|".join(names_pos)}={"|".join(values_pos)}'

            names_ner.append(f'qdelimiter_ner[{index}]')
            values_ner.append(f'{qner[qtext.index(delimiter)]}')
            delimiters_ner = f'{"|".join(names_ner)}={"|".join(values_ner)}'

        features.append(delimiters)
        features.append(delimiters_pos)
        features.append(delimiters_ner)

        names = []
        values = []
        names_pos = []
        values_pos = []
        names_ner = []
        values_ner = []
    
    return '\t'.join(features)

def write_to_disk(crf_lines, output):
    """
        Utility function used to write to disk the formatted data
    """
    
    with open(output, 'a') as bnf_file:
        for line in crf_lines:
            bnf_file.write('\t'.join(line))
            bnf_file.write('\n\n'

In [66]:
for qa_pair in root:
    
    for element in qa_pair:
        print(element.text.split("\n")[1].replace('?', '').strip())
        # candidate = word_tokenize(element.text.split("\n")[1].strip())
        # candidate_tags = pos_tag(candidate)
        # print(' '.join([tag for word, tag in candidate_tags]))
        # print()
        text = len(element.text.split("\n")[1].strip().replace('?', '').split())
        pos = len(element.text.split("\n")[2].strip().split())
        ner = len(element.text.split("\n")[3].strip().split())
        print(f'text: {text}')
        print(f'pos: {pos}')
        print(f'ner: {ner}\n')
    print('-' * 50)

which course has been viewed by MA and MH
text: 9
pos: 9
ner: 9

MA has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

MH has viewed course CS260 Human Computer Interaction and MA has viewed course CS260 Human Computer Interaction
text: 17
pos: 17
ner: 17

ANB has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

--------------------------------------------------
which course has been viewed by SA
text: 7
pos: 7
ner: 7

SA has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

MH has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

HM has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

--------------------------------------------------
which course has been viewed by AHK
text: 7
pos: 7
ner: 7

HM has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

AHK has viewed course CS260 Human Computer Interaction
text: 8
pos: 8
ner: 8

SA enrolled to CS260 Human Computer I

In [64]:
from nltk import word_tokenize, pos_tag

candidate = word_tokenize('AM has viewed course CS260 Human Computer Interaction')
candidate_tags = pos_tag(candidate)
print(' '.join(candidate))
print()
print(' '.join([tag for word, tag in candidate_tags]))
print(len(candidate), len([tag for word, tag in candidate_tags]))

AM has viewed course CS260 Human Computer Interaction

NNP VBZ VBN NN NNP NNP NNP NNP
8 8


In [58]:
len('LogUser LogAction LogAction LogInformation LogResource LogResource LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource  LogResource LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource Other LogUser LogAction LogAction LogInformation LogResource LogResource'.split())

68